In [1]:
import pandas as pd
import numpy as np
import math

In [6]:
## Attributes

window = 3
total_trades = 0
pos_trades = 0
neg_trades = 0

pnl_dict = {}
master_df = pd.DataFrame()

xls = pd.ExcelFile('stocks_data.xlsx')

In [7]:
for i in range(len(xls.sheet_names)):

    df = pd.read_excel('stocks_data.xlsx', sheet_name=xls.sheet_names[i])

    df['Adj Open'] = df['Open'] - df['Open']*(df['Close']-df['Adj Close'])/df['Close']
    df.drop(columns=['Open', 'High', 'Low', 'Close'], inplace=True)
    df.rename(columns = {'Adj Close':'close', 'Adj Open':'open'}, inplace = True)

    # return column
    df['return'] = np.log(df['close']/df['open'])

    # Moving Average volume
    df['mvolume'] = df.Volume.rolling(window).mean()
    df['mvolume'] = df['mvolume'].shift(1)

    # Moving Average price
    df['mprice'] = df.close.rolling(window).mean()
    df['mprice'] = df['mprice'].shift(1)

    df.isna().sum()
    df.dropna(inplace=True)

    # generate signal if current volume below last 3 days of average volume and current price below last 3 days of average price, short sell
    df['signal'] = ((df.Volume < df.mvolume) & (df.close < df.mprice))
    df['shortsignal'] = 0  
    df.loc[df.signal,'shortsignal']= -1  
    df['pnl'] = df['return'] * df.shortsignal.shift(1)
    df.pnl.sum()

    # cumulative PNL
    df['cumpnl'] = df.pnl.cumsum()

    total_trades += len(np.where(df['shortsignal'] == -1)[0])
    pos_trades += len(np.where(df['pnl'] > 0)[0])
    neg_trades += len(np.where(df['pnl'] < 0)[0])

    pnl_dict[xls.sheet_names[i]] = (df['pnl'].sum())*100

    new_df = df.drop(columns=['close', 'Volume', 'open', 'return', 'mvolume', 'mprice', 'signal', 'shortsignal', 'cumpnl'])
    new_df['Date'] = pd.to_datetime(new_df['Date'], errors='coerce')
    new_df = new_df.groupby(pd.Grouper(key='Date', axis=0, freq='y')).sum() 
    new_df
    new_df.rename(columns = {'pnl': xls.sheet_names[i]}, inplace = True)
    

    master_df[xls.sheet_names[i]] = new_df

In [8]:
master_df

,DHANI,BAJAJFINSV,BAJAJHLDNG,VAIBHAVGBL,SOBHA,TANLA,CHOLAFIN,VAKRANGEE,KEI,SUZLON
Date,,,,,,,,,,
2010-12-31,0.668898,0.506703,0.134910,-0.065779,0.531481,1.353099,-0.709336,-0.101585,0.543458,0.536632
2011-12-31,0.897196,0.272309,-0.015824,-1.091625,0.624145,1.824674,-0.415887,-0.421064,0.847641,0.579643
2012-12-31,0.371009,0.104094,-0.170050,-1.209802,-0.156548,0.404764,-1.684226,0.036310,0.729314,0.335165
2013-12-31,0.157367,0.061873,0.041506,-0.514911,0.439542,1.017954,-0.135099,-0.025375,0.674930,1.073209
2014-12-31,0.102120,-0.342653,-0.379389,0.256471,0.228047,0.434588,0.109068,-0.048255,0.398754,0.292229
2015-12-31,-0.036599,0.240394,0.050008,0.450996,0.629111,0.299698,0.066469,0.334665,0.031128,0.200037
2016-12-31,0.949430,0.372749,0.356715,0.568705,0.588322,1.046584,0.289236,0.253979,0.160363,0.815912
2017-12-31,0.207780,0.083394,0.100481,-0.246911,0.129864,0.739474,-0.170661,-0.172123,0.053553,0.529840
2018-12-31,-0.161038,-0.054631,0.182181,0.508208,0.561945,0.338272,0.091267,-0.273100,0.148164,0.577645


In [9]:
pnl_dict

{'DHANI': 515.7370897196193,
 'BAJAJFINSV': 156.24490215217608,
 'BAJAJHLDNG': 113.29236562496096,
 'VAIBHAVGBL': -3.067636040802091,
 'SOBHA': 542.422117929037,
 'TANLA': 832.4258016421103,
 'CHOLAFIN': -176.024856845311,
 'VAKRANGEE': 182.65144137778216,
 'KEI': 458.8089770431796,
 'SUZLON': 624.1663697315463}

In [10]:
total_trades

10311

In [11]:
pos_trades

5762

In [12]:
neg_trades

4206

In [13]:
master_df

,DHANI,BAJAJFINSV,BAJAJHLDNG,VAIBHAVGBL,SOBHA,TANLA,CHOLAFIN,VAKRANGEE,KEI,SUZLON
Date,,,,,,,,,,
2010-12-31,0.668898,0.506703,0.134910,-0.065779,0.531481,1.353099,-0.709336,-0.101585,0.543458,0.536632
2011-12-31,0.897196,0.272309,-0.015824,-1.091625,0.624145,1.824674,-0.415887,-0.421064,0.847641,0.579643
2012-12-31,0.371009,0.104094,-0.170050,-1.209802,-0.156548,0.404764,-1.684226,0.036310,0.729314,0.335165
2013-12-31,0.157367,0.061873,0.041506,-0.514911,0.439542,1.017954,-0.135099,-0.025375,0.674930,1.073209
2014-12-31,0.102120,-0.342653,-0.379389,0.256471,0.228047,0.434588,0.109068,-0.048255,0.398754,0.292229
2015-12-31,-0.036599,0.240394,0.050008,0.450996,0.629111,0.299698,0.066469,0.334665,0.031128,0.200037
2016-12-31,0.949430,0.372749,0.356715,0.568705,0.588322,1.046584,0.289236,0.253979,0.160363,0.815912
2017-12-31,0.207780,0.083394,0.100481,-0.246911,0.129864,0.739474,-0.170661,-0.172123,0.053553,0.529840
2018-12-31,-0.161038,-0.054631,0.182181,0.508208,0.561945,0.338272,0.091267,-0.273100,0.148164,0.577645


In [14]:
master_df['profit_total_year'] = master_df.sum(axis=1)*100

In [15]:
new_df = master_df.copy()

In [16]:
master_df.loc[len(master_df.index)] = master_df.sum(axis=0)
master_df.rename(index={13: 'company_total_profit'}, inplace=True)

In [17]:
master_df

,DHANI,BAJAJFINSV,BAJAJHLDNG,VAIBHAVGBL,SOBHA,TANLA,CHOLAFIN,VAKRANGEE,KEI,SUZLON,profit_total_year
Date,,,,,,,,,,,
2010-12-31 00:00:00,0.668898,0.506703,0.134910,-0.065779,0.531481,1.353099,-0.709336,-0.101585,0.543458,0.536632,339.848125
2011-12-31 00:00:00,0.897196,0.272309,-0.015824,-1.091625,0.624145,1.824674,-0.415887,-0.421064,0.847641,0.579643,310.120830
2012-12-31 00:00:00,0.371009,0.104094,-0.170050,-1.209802,-0.156548,0.404764,-1.684226,0.036310,0.729314,0.335165,-123.997126
2013-12-31 00:00:00,0.157367,0.061873,0.041506,-0.514911,0.439542,1.017954,-0.135099,-0.025375,0.674930,1.073209,279.099560
2014-12-31 00:00:00,0.102120,-0.342653,-0.379389,0.256471,0.228047,0.434588,0.109068,-0.048255,0.398754,0.292229,105.098037
2015-12-31 00:00:00,-0.036599,0.240394,0.050008,0.450996,0.629111,0.299698,0.066469,0.334665,0.031128,0.200037,226.590562
2016-12-31 00:00:00,0.949430,0.372749,0.356715,0.568705,0.588322,1.046584,0.289236,0.253979,0.160363,0.815912,540.199670
2017-12-31 00:00:00,0.207780,0.083394,0.100481,-0.246911,0.129864,0.739474,-0.170661,-0.172123,0.053553,0.529840,125.469033
2018-12-31 00:00:00,-0.161038,-0.054631,0.182181,0.508208,0.561945,0.338272,0.091267,-0.273100,0.148164,0.577645,191.891237


In [18]:
master_df.to_csv('master.csv')

In [19]:
new_df

,DHANI,BAJAJFINSV,BAJAJHLDNG,VAIBHAVGBL,SOBHA,TANLA,CHOLAFIN,VAKRANGEE,KEI,SUZLON,profit_total_year
Date,,,,,,,,,,,
2010-12-31,0.668898,0.506703,0.134910,-0.065779,0.531481,1.353099,-0.709336,-0.101585,0.543458,0.536632,339.848125
2011-12-31,0.897196,0.272309,-0.015824,-1.091625,0.624145,1.824674,-0.415887,-0.421064,0.847641,0.579643,310.120830
2012-12-31,0.371009,0.104094,-0.170050,-1.209802,-0.156548,0.404764,-1.684226,0.036310,0.729314,0.335165,-123.997126
2013-12-31,0.157367,0.061873,0.041506,-0.514911,0.439542,1.017954,-0.135099,-0.025375,0.674930,1.073209,279.099560
2014-12-31,0.102120,-0.342653,-0.379389,0.256471,0.228047,0.434588,0.109068,-0.048255,0.398754,0.292229,105.098037
2015-12-31,-0.036599,0.240394,0.050008,0.450996,0.629111,0.299698,0.066469,0.334665,0.031128,0.200037,226.590562
2016-12-31,0.949430,0.372749,0.356715,0.568705,0.588322,1.046584,0.289236,0.253979,0.160363,0.815912,540.199670
2017-12-31,0.207780,0.083394,0.100481,-0.246911,0.129864,0.739474,-0.170661,-0.172123,0.053553,0.529840,125.469033
2018-12-31,-0.161038,-0.054631,0.182181,0.508208,0.561945,0.338272,0.091267,-0.273100,0.148164,0.577645,191.891237


In [20]:
abs_total_return = round(new_df.profit_total_year.sum(), 3)

In [21]:
avg_yearly_return = round(new_df.profit_total_year.mean(), 3)

In [22]:
cagr = round((pow((abs_total_return + 10)/10, 1/12.5)-1)*100, 3)

In [23]:
print(abs_total_return, avg_yearly_return, cagr)

3246.657 249.743 58.863


In [24]:
std_pnl = new_df.profit_total_year.std()
std_pnl

160.4043422031439

In [25]:
sharpe_ratio = round(avg_yearly_return / std_pnl, 3)
sharpe_ratio

1.557